In [2]:
import numpy as np
import pandas as pd
import xarray as xr
from joblib import dump, load
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import glob

import matplotlib as mpl

import matplotlib.pyplot as plt

import math as mt

import cartopy.crs as ccrs

import cartopy.feature as cfeature

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

#AO Calculation

In [3]:
# Calculate AO - input std normal latitude wighted 1000mb z
ZData = xr.open_dataset('../../../Data/CMIP6/INM-CM4-8/processed/weighted1000Std.nc')
#SLPData = xr.open_dataset('../eraDown/ERA5_2degree_Down/DailyMean/weighted1000.nc')

ZData.zg.values
ZData.zg.values=xr.where(np.isnan(ZData.zg.values),  0.000000000001,ZData.zg.values)
ZData.zg.values

array([[[ 0.12850985,  0.07573885, -0.01635843, ...,  0.1599361 ,
          0.14527868,  0.14055571],
        [ 0.10198791,  0.03961104, -0.07666926, ...,  0.15636194,
          0.1507506 ,  0.13183261],
        [ 0.12693352,  0.05379534, -0.04674144, ...,  0.1779603 ,
          0.19145924,  0.17172174],
        ...,
        [ 0.03234478,  0.04210443,  0.05173143, ...,  0.00315356,
          0.0127667 ,  0.02253194],
        [ 0.05852018,  0.06111548,  0.06368198, ...,  0.05073507,
          0.05331065,  0.05591204],
        [ 0.04880805,  0.04876667,  0.04871674, ...,  0.04889037,
          0.04887085,  0.04884258]],

       [[ 0.9102813 ,  0.7908809 ,  0.65494883, ...,  0.848859  ,
          0.9307297 ,  0.95032096],
        [ 0.95002586,  0.8759926 ,  0.75123405, ...,  0.91998374,
          0.9733444 ,  0.97348785],
        [ 1.0134833 ,  0.9863147 ,  0.95354015, ...,  0.9731936 ,
          1.0147485 ,  1.026888  ],
        ...,
        [ 0.02359757,  0.02612042,  0.02958885, ...,  

In [4]:


t1=ZData.zg.stack(z=("lat", "lon"))
# fit scaler on training data
zg_PC =PCA().fit_transform(t1)


In [5]:
print(zg_PC[:,0].shape)
AO_ds = xr.Dataset({'AO': (('time'), zg_PC[:,0])}, coords={'time': ZData.time})


norm = StandardScaler().fit(zg_PC[:,0].reshape(-1, 1))
    # transform training data
aoX = norm.transform(zg_PC[:,0].reshape(-1, 1))
aoX.shape

AO_ds = xr.Dataset({'AO': (('time'), aoX[:,0]*-1.0)}, coords={'time': ZData.time})


(4800,)


In [6]:
AO_ds.to_netcdf('INM-CM4-8-AOindex-NDJF-Daily-1980-2014.nc')

In [7]:
AO_ds.AO.where(AO_ds.AO==AO_ds.AO.max(), drop=True).squeeze()

<xarray.DataArray 'AO' ()>
array(3.240556, dtype=float32)
Coordinates:
    time     object 1994-11-29 12:00:00

In [8]:
#AO_ds.AO.where(AO_ds.AO>2.0, drop=True).squeeze()

In [9]:
AO_ds.AO.where(AO_ds.AO==AO_ds.AO.min(), drop=True).squeeze()

<xarray.DataArray 'AO' ()>
array(-3.3191388, dtype=float32)
Coordinates:
    time     object 1997-01-16 12:00:00

In [10]:
#AO_ds.AO.where(AO_ds.AO<-2, drop=True).squeeze()

#EU Calculation

In [11]:
zLevData = xr.open_dataset('../../../Data/CMIP6/INM-CM4-8/processed/zg_NDJFday_INM-CM4-8_historical_r1i1p1f1_gr1_19750101-20141231.nc')

In [12]:
zLevData.plev

<xarray.DataArray 'plev' (plev: 8)>
array([100000.,  85000.,  70000.,  50000.,  25000.,  10000.,   5000.,   1000.])
Coordinates:
  * plev     (plev) float64 1e+05 8.5e+04 7e+04 5e+04 2.5e+04 1e+04 5e+03 1e+03
Attributes:
    standard_name:  air_pressure
    long_name:      pressure
    units:          Pa
    positive:       down
    axis:           Z

In [13]:
z500=zLevData.zg.sel(plev=50000.0,method='nearest')
z500

<xarray.DataArray 'zg' (time: 4800, lat: 120, lon: 180)>
[103680000 values with dtype=float32]
Coordinates:
  * time     (time) object 1975-01-01 12:00:00 ... 2014-12-31 12:00:00
  * lon      (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float64 -89.25 -87.75 -86.25 -84.75 ... 86.25 87.75 89.25
    plev     float64 5e+04
Attributes:
    standard_name:  geopotential_height
    long_name:      Geopotential Height
    units:          m
    comment:        Geopotential is the sum of the specific gravitational pot...
    original_name:  zg
    cell_methods:   time: mean (interval: 1 day)
    cell_measures:  area: areacella

In [14]:
z145E40N=z500.sel(lat=40,lon=145,method='nearest')

In [15]:
#z145E40N

In [16]:
z20E55N=z500.sel(lat=55,lon=20,method='nearest')

In [17]:
#z20E55N

In [18]:
z75E55N=z500.sel(lat=55,lon=75,method='nearest')

In [19]:
#z75E55N

In [20]:
#Calculate Anomaly
z145E40N_Anom = z145E40N.groupby("time.dayofyear") - z145E40N.groupby("time.dayofyear").mean("time")
z75E55N_Anom  = z75E55N.groupby("time.dayofyear")  - z75E55N.groupby("time.dayofyear").mean("time")
z20E55N_Anom  = z20E55N.groupby("time.dayofyear")  - z20E55N.groupby("time.dayofyear").mean("time")

In [21]:
# calculate std normal anomalies
z145E40N_AnomStd = z145E40N_Anom.groupby("time.dayofyear")  / z145E40N.groupby("time.dayofyear").std("time")
z75E55N_AnomStd  = z75E55N_Anom.groupby("time.dayofyear")   / z75E55N.groupby("time.dayofyear").std("time")
z20E55N_AnomStd  = z20E55N_Anom.groupby("time.dayofyear")   / z20E55N.groupby("time.dayofyear").std("time")

In [22]:
EUVal = -(1.0/4.0)*z20E55N_AnomStd + (1.0/2.0)*z75E55N_AnomStd - (1.0/4.0)*z145E40N_AnomStd

In [23]:
EUVal

<xarray.DataArray 'zg' (time: 4800)>
array([ 0.88493193,  0.73412334,  1.63229492, ..., -0.47142787,
        0.06978158,  0.63036225])
Coordinates:
  * time       (time) object 1975-01-01 12:00:00 ... 2014-12-31 12:00:00
    plev       float64 5e+04
    dayofyear  (time) int64 1 2 3 4 5 6 7 8 ... 358 359 360 361 362 363 364 365
    lon        float64 146.0

In [24]:
EU_ds = xr.Dataset({'EU': (('time'), EUVal)}, coords={'time': zLevData.time})

In [25]:
EU_ds.to_netcdf('INM-CM4-8-EUindex-NDJF-Daily-1980-2014.nc')